In [2]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('^N225', 'yahoo', start, end)
data = data.astype("double")
print(data)

initial = 1000000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

             High    Low   Open  Close       Volume  Adj Close
Date                                                          
2021-09-14  363.0  356.0  361.0  358.0   66934700.0      358.0
2021-09-15  367.0  361.0  364.0  362.0   54868371.0      362.0
2021-09-16  371.0  360.0  360.0  367.0   52270565.0      367.0
2021-09-17  367.0  362.0  366.0  362.0   43735573.0      362.0
2021-09-21  379.0  373.0  378.0  378.0   83213264.0      378.0
...           ...    ...    ...    ...          ...        ...
2022-09-08  367.0  357.0  367.0  357.0  113510594.0      357.0
2022-09-09  357.0  352.0  354.0  353.0   99795902.0      353.0
2022-09-12  348.0  343.0  346.0  346.0   73706891.0      346.0
2022-09-13  346.0  342.0  345.0  344.0   46433034.0      344.0
2022-09-14  364.0  359.0  363.0  359.0   79050084.0      359.0

[246 rows x 6 columns]


**カスタムOptimization**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize=opt.optim_func,\
    method='grid',\
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot()

**MaxTries回数を設定**
**Constraintを設定**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output2=bt.optimize(\
    upper_bound = range(10, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 85, 5),     #Constraintのお陰で変な値にならない
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot()

**Heat Map**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output2, heatmap=bt.optimize(\
    upper_bound = range(10, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 85, 5),     #Constraintのお陰で変な値にならない
    rsi_window=14,\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 1000,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)

print(heatmap)
hml = heatmap.groupby(["upper_bound", "lower_bound"]).mean().unstack()
sns.heatmap(hml)
plt.show()
# bt.plot()

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 1000,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)

print(heatmap)
plot_heatmaps(heatmap, agg = "mean")

**時系列データ resamply_apply**

In [ ]:
#週平均のRSIが上限下限を超えたとき、という条件を追加
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_WithWeekly, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") 


# output2=bt.optimize(\
#     upper_bound = range(50, 85, 5),
#     lower_bound = range(10, 45, 5),
#     rsi_window=range(10, 30, 2),\
#     maximize='Equity Final [$]',\
#     method='grid',\
#     max_tries = 1000)
# print(output2)
# display(output2._trades)
# display(output2._strategy)
# bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

**空売りあり**

In [ ]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + ".html") 

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 1000,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

**損切の設定**

In [86]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_WithStopLoss, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + ".html") 

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 1000,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    7.317073
Equity Final [$]                1000005.32195
Equity Peak [$]                 1000033.32195
Return [%]                           0.000532
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                    0.000545
Volatility (Ann.) [%]                0.003289
Sharpe Ratio                         0.165737
Sortino Ratio                        0.247745
Calmar Ratio                         0.194712
Max. Drawdown [%]                     -0.0028
Avg. Drawdown [%]                   -0.000954
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       1.562164
Worst Trade [%]                      1.562164
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,227,244,340.67805,346.0,5.32195,0.015622,2022-08-18,2022-09-12,25 days


<Strategy RsiOscillator_WithStopLoss(upper_bound=65,lower_bound=30,rsi_window=10)>

Start                     2021-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   23.577236
Equity Final [$]                1000093.03555
Equity Peak [$]                 1000115.54095
Return [%]                           0.009304
Buy & Hold Return [%]               -5.753425
Return (Ann.) [%]                     0.00953
Volatility (Ann.) [%]                0.006887
Sharpe Ratio                         1.383808
Sortino Ratio                         2.45013
Calmar Ratio                         2.006421
Max. Drawdown [%]                    -0.00475
Avg. Drawdown [%]                   -0.001325
Max. Drawdown Duration       42 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                   10
Win Rate [%]                             70.0
Best Trade [%]                       9.384107
Worst Trade [%]                     -5.467934
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,37,44,377.86120,371.00,-6.86120,-0.018158,2021-11-08,2021-11-17,9 days
1,1,44,48,372.83645,380.00,7.16355,0.019214,2021-11-17,2021-11-24,7 days
2,1,48,50,381.88100,395.00,13.11900,0.034354,2021-11-24,2021-11-26,2 days
3,1,77,85,394.94535,421.00,26.05465,0.065970,2022-01-06,2022-01-19,13 days
4,1,137,141,398.96515,429.00,30.03485,0.075282,2022-04-06,2022-04-12,6 days
5,1,149,159,404.99485,443.00,38.00515,0.093841,2022-04-22,2022-05-12,20 days
6,1,181,182,400.97505,409.00,8.02495,0.020014,2022-06-13,2022-06-14,1 days
7,1,216,220,376.85625,356.25,-20.60625,-0.054679,2022-08-02,2022-08-08,6 days
8,1,222,224,368.81665,348.65,-20.16665,-0.054679,2022-08-10,2022-08-15,5 days
9,1,230,238,351.73250,370.00,18.26750,0.051936,2022-08-23,2022-09-02,10 days


<Strategy RsiOscillator_WithStopLoss(upper_bound=60,lower_bound=40,rsi_window=10)>

Row(id='84272', ...)

**単位買い（シグナルごと1株ずつ買う等、ドルコスト平均法とか）**

In [72]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_BuySize, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + ".html") 

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 1000,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    7.346939
Equity Final [$]                 1000019.3566
Equity Peak [$]                  1000040.3566
Return [%]                           0.001936
Buy & Hold Return [%]               -5.205479
Return (Ann.) [%]                    0.001991
Volatility (Ann.) [%]                0.003257
Sharpe Ratio                         0.611215
Sortino Ratio                        0.957121
Calmar Ratio                         0.948117
Max. Drawdown [%]                     -0.0021
Avg. Drawdown [%]                   -0.000975
Max. Drawdown Duration        5 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       5.801583
Worst Trade [%]                      5.801583
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,226,243,333.6434,353.0,19.3566,0.058016,2022-08-17,2022-09-09,23 days


<Strategy RsiOscillator_BuySize(upper_bound=55,lower_bound=40,rsi_window=20)>

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   30.204082
Equity Final [$]                 1000078.0812
Equity Peak [$]                 1000105.09555
Return [%]                           0.007808
Buy & Hold Return [%]               -5.205479
Return (Ann.) [%]                    0.008031
Volatility (Ann.) [%]                0.007503
Sharpe Ratio                         1.070404
Sortino Ratio                        1.735294
Calmar Ratio                         1.732132
Max. Drawdown [%]                   -0.004637
Avg. Drawdown [%]                   -0.001582
Max. Drawdown Duration       41 days 00:00:00
Avg. Drawdown Duration       14 days 00:00:00
# Trades                                   11
Win Rate [%]                        27.272727
Best Trade [%]                       15.70025
Worst Trade [%]                     -7.561396
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,127,128,387.91070,384.0,-3.91070,-0.010081,2022-03-23,2022-03-24,1 days
1,1,128,129,385.90080,383.0,-2.90080,-0.007517,2022-03-24,2022-03-25,1 days
2,1,129,131,384.89585,381.0,-3.89585,-0.010122,2022-03-25,2022-03-29,4 days
3,1,131,132,382.88595,381.0,-1.88595,-0.004926,2022-03-29,2022-03-30,1 days
4,1,132,159,382.88595,443.0,60.11405,0.157002,2022-03-30,2022-05-12,43 days
5,1,178,179,366.80675,365.0,-1.80675,-0.004926,2022-06-08,2022-06-09,1 days
6,1,179,183,366.80675,419.0,52.19325,0.142291,2022-06-09,2022-06-15,6 days
7,1,209,224,367.81170,340.0,-27.81170,-0.075614,2022-07-22,2022-08-15,24 days
8,1,224,225,341.68300,341.0,-0.68300,-0.001999,2022-08-15,2022-08-16,1 days
9,1,225,226,342.68795,332.0,-10.68795,-0.031189,2022-08-16,2022-08-17,1 days


<Strategy RsiOscillator_BuySize(upper_bound=65,lower_bound=30,rsi_window=10)>

Row(id='57224', ...)

**●●の指標が××から何日間経過したか**

In [74]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

import BacktestingStrategies.Optimization as opt
importlib.reload(opt)

import seaborn as sns
import matplotlib.pyplot as plt

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator_BarsSince, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + ".html") 

output2, heatmap=bt.optimize(\
    upper_bound = range(55, 85, 5),     #Constraintのお陰で変な値にならない
    lower_bound = range(10, 45, 5),     #Constraintのお陰で変な値にならない
    rsi_window= range(10, 45, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: x.upper_bound > x.lower_bound,\
    max_tries = 1000,
    return_heatmap=True)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    7.346939
Equity Final [$]                 1000019.3566
Equity Peak [$]                  1000040.3566
Return [%]                           0.001936
Buy & Hold Return [%]               -5.205479
Return (Ann.) [%]                    0.001991
Volatility (Ann.) [%]                0.003257
Sharpe Ratio                         0.611215
Sortino Ratio                        0.957121
Calmar Ratio                         0.948117
Max. Drawdown [%]                     -0.0021
Avg. Drawdown [%]                   -0.000975
Max. Drawdown Duration        5 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       5.801583
Worst Trade [%]                      5.801583
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,226,243,333.6434,353.0,19.3566,0.058016,2022-08-17,2022-09-09,23 days


<Strategy RsiOscillator_BarsSince(upper_bound=65,lower_bound=25,rsi_window=10)>

Start                     2021-09-13 00:00:00
End                       2022-09-12 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    7.346939
Equity Final [$]                 1000019.3566
Equity Peak [$]                  1000040.3566
Return [%]                           0.001936
Buy & Hold Return [%]               -5.205479
Return (Ann.) [%]                    0.001991
Volatility (Ann.) [%]                0.003257
Sharpe Ratio                         0.611215
Sortino Ratio                        0.957121
Calmar Ratio                         0.948117
Max. Drawdown [%]                     -0.0021
Avg. Drawdown [%]                   -0.000975
Max. Drawdown Duration        5 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       5.801583
Worst Trade [%]                      5.801583
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,226,243,333.6434,353.0,19.3566,0.058016,2022-08-17,2022-09-09,23 days


<Strategy RsiOscillator_BarsSince(upper_bound=55,lower_bound=30,rsi_window=15)>

Row(id='62040', ...)

**ボリンジャーバンドとRSI組み合わせ**

In [3]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.BBandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.BBandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   42.276423
Equity Final [$]                  1000496.261
Equity Peak [$]                  1225630.8635
Return [%]                           0.049626
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                    0.050837
Volatility (Ann.) [%]               24.867373
Sharpe Ratio                         0.002044
Sortino Ratio                        0.003124
Calmar Ratio                         0.002036
Max. Drawdown [%]                  -24.969797
Avg. Drawdown [%]                   -6.865166
Max. Drawdown Duration       86 days 00:00:00
Avg. Drawdown Duration       25 days 00:00:00
# Trades                                    8
Win Rate [%]                             50.0
Best Trade [%]                       9.558695
Worst Trade [%]                     -9.829721
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2512,127,159,397.96020,436.0,95555.97760,0.095587,2022-03-24,2022-05-13,50 days
1,2838,175,176,385.90080,372.0,-39450.47040,-0.036022,2022-06-06,2022-06-07,1 days
2,2825,176,178,373.84140,366.0,-22151.95500,-0.020975,2022-06-07,2022-06-09,2 days
3,2811,178,207,367.81170,375.0,20206.31130,0.019543,2022-06-09,2022-07-21,42 days
4,2797,207,208,376.85625,370.0,-19176.93125,-0.018193,2022-07-21,2022-07-22,1 days
5,2783,208,209,371.83150,373.0,3251.93550,0.003143,2022-07-22,2022-07-25,3 days
6,2769,209,226,374.84635,338.0,-102027.54315,-0.098297,2022-07-25,2022-08-18,24 days
7,2756,226,245,339.67310,363.0,64288.93640,0.068675,2022-08-18,2022-09-14,27 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   42.682927
Equity Final [$]                 1310968.5235
Equity Peak [$]                  1310968.5235
Return [%]                          31.096852
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                   31.965488
Volatility (Ann.) [%]               32.368294
Sharpe Ratio                         0.987556
Sortino Ratio                         2.24024
Calmar Ratio                         2.346235
Max. Drawdown [%]                  -13.624164
Avg. Drawdown [%]                   -4.699335
Max. Drawdown Duration       56 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                   16
Win Rate [%]                             50.0
Best Trade [%]                      13.113013
Worst Trade [%]                     -8.573212
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2646,33,42,377.86120,370.0,-20800.73520,-0.020804,2021-11-02,2021-11-16,14 days
1,2633,42,50,371.83150,403.0,82066.66050,0.083824,2021-11-16,2021-11-29,13 days
2,2831,75,85,374.84635,424.0,139153.98315,0.131130,2022-01-05,2022-01-20,15 days
3,2771,124,127,433.13345,396.0,-102896.78995,-0.085732,2022-03-18,2022-03-24,6 days
4,2757,127,141,397.96020,425.0,74548.72860,0.067946,2022-03-24,2022-04-13,20 days
5,2894,148,159,404.99485,436.0,89728.90410,0.076557,2022-04-22,2022-05-13,21 days
6,3211,171,175,392.93545,384.0,-28691.72995,-0.022740,2022-05-31,2022-06-06,6 days
7,3195,175,176,385.90080,372.0,-44413.05600,-0.036022,2022-06-06,2022-06-07,1 days
8,3179,176,178,373.84140,366.0,-24927.81060,-0.020975,2022-06-07,2022-06-09,2 days
9,3164,178,182,367.81170,410.0,133483.78120,0.114701,2022-06-09,2022-06-15,6 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2537,51,127,394.0398,396.0,-4973.0274,-0.004975,2021-11-30,2022-03-24,114 days
1,2500,127,159,397.9602,436.0,95099.5000,0.095587,2022-03-24,2022-05-13,50 days
2,-2512,159,175,433.8418,384.0,125202.6016,0.114885,2022-05-13,2022-06-06,24 days
3,3149,175,245,385.9008,363.0,-72114.6192,-0.059344,2022-06-06,2022-09-14,100 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   86.585366
Equity Final [$]                1900419.70255
Equity Peak [$]                 1900419.70255
Return [%]                           90.04197
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                   93.041522
Volatility (Ann.) [%]                80.18508
Sharpe Ratio                         1.160335
Sortino Ratio                        3.788357
Calmar Ratio                         4.171789
Max. Drawdown [%]                  -22.302549
Avg. Drawdown [%]                   -4.879621
Max. Drawdown Duration       70 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                   13
Win Rate [%]                        76.923077
Best Trade [%]                      13.113013
Worst Trade [%]                     -2.156619
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2646,33,50,377.86120,403.0,66517.26480,0.066529,2021-11-02,2021-11-29,27 days
1,-2659,50,75,401.00515,373.0,74465.69385,0.069837,2021-11-29,2022-01-05,37 days
2,3043,75,85,374.84635,424.0,149574.55695,0.131130,2022-01-05,2022-01-20,15 days
3,-3058,85,124,421.90120,431.0,-27824.13040,-0.021566,2022-01-20,2022-03-18,57 days
4,2915,124,141,433.13345,425.0,-23709.00675,-0.018778,2022-03-18,2022-04-13,26 days
5,-2929,141,148,422.89625,403.0,58276.11625,0.047048,2022-04-13,2022-04-22,9 days
6,3203,148,159,404.99485,436.0,99309.49545,0.076557,2022-04-22,2022-05-13,21 days
7,-3219,159,171,433.84180,391.0,137907.75420,0.098750,2022-05-13,2022-05-31,18 days
8,3905,171,182,392.93545,410.0,66637.06775,0.043428,2022-05-31,2022-06-15,15 days
9,-3924,182,207,407.97050,375.0,129376.24200,0.080816,2022-06-15,2022-07-21,36 days


<Strategy BBandRSI_WithShortPosition(upper_bound=60,lower_bound=40,rsi_window=10)>